In [ ]:
import os
import cv2
import csv
import yaml
from roboflow import Roboflow
from ultralytics import YOLO
from collections import defaultdict
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
detection_model = YOLO('detection_best.pt')     
classification_model = YOLO('classifier_best.pt')  
class_map = {
    0: "major_pothole",
    1: "medium_pothole",
    2: "background"
}

def add_context_padding(x1, y1, x2, y2, img_w, img_h, pad_ratio=0.2):
    box_w, box_h = x2 - x1, y2 - y1
    pad_w, pad_h = int(box_w * pad_ratio), int(box_h * pad_ratio)
    x1_p = max(x1 - pad_w, 0)
    y1_p = max(y1 - pad_h, 0)
    x2_p = min(x2 + pad_w, img_w)
    y2_p = min(y2 + pad_h, img_h)
    return x1_p, y1_p, x2_p, y2_p

def draw_box(img, bbox, label, color=(0, 255, 0), thickness=2):
    x1, y1, x2, y2 = bbox
    cv2.rectangle(img, (x1, y1), (x2, y2), color, thickness)
    cv2.putText(img, label, (x1, max(y1 - 10, 0)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

def detect_and_classify_image(image_path, output_dir, context_padding=0.2):
    image_name = os.path.basename(image_path)
    base_name = os.path.splitext(image_name)[0]

    image = cv2.imread(image_path)
    if image is None:
        return []

    draw_image = image.copy()
    img_h, img_w = image.shape[:2]

    detection_results = detection_model(image_path)[0]
    results = []

    for i, box in enumerate(detection_results.boxes):
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        x1_p, y1_p, x2_p, y2_p = add_context_padding(x1, y1, x2, y2, img_w, img_h, pad_ratio=context_padding)

        cropped = image[y1_p:y2_p, x1_p:x2_p]
        temp_crop_path = os.path.join(output_dir, "temp.jpg")
        cv2.imwrite(temp_crop_path, cropped)
        class_result = classification_model(temp_crop_path)[0]
        os.remove(temp_crop_path)

        if class_result.probs is not None:
            class_id = int(class_result.probs.top1)
            label_name = class_map.get(class_id, "unknown")
        else:
            class_id = -1
            label_name = "unclassified"

        draw_box(draw_image, [x1, y1, x2, y2], label_name)

        results.append({
            "image": image_name,
            "bbox": [x1, y1, x2, y2],
            "padded_bbox": [x1_p, y1_p, x2_p, y2_p],
            "label": label_name,
            "class_id": class_id
        })
        
    cv2.imwrite(os.path.join(output_dir, f"{base_name}_annotated.jpg"), draw_image)
    return results

def run_pipeline_on_folder(image_folder, output_dir, results_csv):
    os.makedirs(output_dir, exist_ok=True)

    with open(results_csv, 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=["image", "bbox", "padded_bbox", "label", "class_id"])
        writer.writeheader()

        for filename in os.listdir(image_folder):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(image_folder, filename)
                print(f"Processing {filename} ...")
                result = detect_and_classify_image(image_path, output_dir)
                for r in result:
                    writer.writerow(r)


In [ ]:
def compute_iou(box1, box2):
    xA = max(box1[0], box2[0])
    yA = max(box1[1], box2[1])
    xB = min(box1[2], box2[2])
    yB = min(box1[3], box2[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)
    box1Area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2Area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    unionArea = box1Area + box2Area - interArea

    return interArea / unionArea if unionArea > 0 else 0

def evaluate_detection_and_classification(results_by_image, label_dir, image_dir, iou_thresh=0.2):
    TP_det = 0
    FP_det = 0
    FN_det = 0
    
    correct_classifications = 0
    y_true = []
    y_pred = []
    
    class_names = {
        0: "major_pothole",
        1: "medium_pothole", 
        2: "background"
    }

    for image_name, preds in results_by_image.items():
        label_path = os.path.join(label_dir, f"{os.path.splitext(image_name)[0]}.txt")
        img = cv2.imread(os.path.join(image_dir, image_name))
        
        if img is None:
            continue
            
        H, W = img.shape[:2]
        gt_boxes = []
        
        if os.path.exists(label_path):
            with open(label_path) as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) == 5:
                        cls, x, y, w, h = map(float, parts)
                        x1 = int((x - w/2) * W)
                        y1 = int((y - h/2) * H)
                        x2 = int((x + w/2) * W)
                        y2 = int((y + h/2) * H)
                        gt_boxes.append({
                            'bbox': [x1, y1, x2, y2],
                            'cls': int(cls),
                            'matched': False
                        })

        for pred in preds:
            pred_box = pred['bbox']
            pred_cls = pred['class_id']
            best_iou = 0
            best_gt = None

            for gt in gt_boxes:
                if gt['matched']:
                    continue
                iou = compute_iou(pred_box, gt['bbox'])
                if iou > best_iou:
                    best_iou = iou
                    best_gt = gt

            if best_iou >= iou_thresh and best_gt:
                TP_det += 1
                best_gt['matched'] = True
                if pred_cls == best_gt['cls']:
                    correct_classifications += 1
                y_true.append(best_gt['cls'])
                y_pred.append(pred_cls)
            else:
                FP_det += 1
                y_true.append(2)  
                y_pred.append(pred_cls)
        for gt in gt_boxes:
            if not gt['matched']:
                FN_det += 1
                y_true.append(gt['cls'])
                y_pred.append(2)  
    precision = TP_det / (TP_det + FP_det + 1e-6)
    recall = TP_det / (TP_det + FN_det + 1e-6)
    f1_score = 2 * (precision * recall) / (precision + recall + 1e-6)
    
    print("\n Detection Metrics:")
    print(f"True Positives (TP): {TP_det}")
    print(f"False Positives (FP): {FP_det}") 
    print(f"False Negatives (FN): {FN_det}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1_score:.4f}")
    cls_acc = correct_classifications / (TP_det + 1e-6)
    
    print("\n Classification Metrics:")
    print(f"Accuracy: {cls_acc:.4f} (on matched detections)")
    
    cm = confusion_matrix(y_true, y_pred, labels=[0,1,2])
    report = classification_report(
        y_true, y_pred, 
        target_names=[class_names[i] for i in [0,1,2]],
        digits=4
    )
    print("\nClassification Report:")
    print(report)

    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=[class_names[i] for i in [0,1,2]],
                yticklabels=[class_names[i] for i in [0,1,2]])
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.tight_layout()
    plt.savefig("pipeline_detection_classification_results.png")
    plt.show()

In [ ]:
rf = Roboflow(api_key="vJRRNV9HbDPFy95m0veX")
project = rf.workspace("object-detection-pxcal").project("test_set-with-added-images")
version = project.version(1)
dataset = version.download("yolov11")
                

In [ ]:
base_dire = f"test_set-(With-Added-Images)-1\\test"
image_folder=f"{base_dire}\images"
output_dir = "test_output_pipeline"

results_by_image = {}
for filename in os.listdir(image_folder):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(image_folder, filename)
                print(image_path)
                results = detect_and_classify_image(image_path, output_dir)
                results_by_image[filename] = results
evaluate_detection_and_classification(
    results_by_image=results_by_image,
    label_dir=f"{base_dire}\labels",
    image_dir=f"{base_dire}\images"
)